# universal function

除了通用的序列操作外,还支持`ufunc`
`ufunc`是`universal function`的简写,它是一种对数组中每个元素做相同操作的函数,概念上类似原生python的`map`,但在实际的运算中又不同.
原生map实际上是运行迭代器一个一个操作,而`universal function`则是向量化的执行函数,即一个函数不同的数据一起运行,这样就大大提高了效率.

我们可以实际测试下python自带的迭代和universal function的性能差距

In [ ]:
test = np.arange(int(1e5))
print(test)

In [ ]:
%timeit -n 3 map(lambda x:x**2,test)

In [ ]:
%timeit -n 3 test**2

## 广播

可见universal function的高效.universal function的另一个特性是两个ndarray对象可以对应项计算,这一特性被称作广播

当俩数组形状不同的时候,那就会进行广播处理

1. 让所有数组向其中维数最多的数组看齐,shape不足的部分通过在前面加1补齐
2. 输出数组的shape属性是输入数组shape属性在各轴上的最大值
3. 如果输入数组的某个轴长度是1或输出数组对应数组对应轴的长度相同,这个数组就够用来计算,否则出错
4. 当输入数组的某个轴长度为1时,沿着该轴运算时都用此轴上的额第一组值

看例子:

### 相同shape

In [ ]:
a = np.arange(12).reshape(3,4)
a

In [ ]:
b = np.arange(3,15).reshape(3,4)
b

In [ ]:
a+b

### 不同shape在对应轴上广播

In [ ]:
a = np.arange(0,60,10).reshape(-1,1)
print(a)
a.shape

In [ ]:
b = np.arange(0,5)
print(b)
b.shape

In [ ]:
c = a+b
print(c)
c.shape

## 与常量做四则运算

ndarray对象支持python支持的四则运算接口,包括幂运算,而且都是universal function

In [ ]:
X = np.arange(10).reshape(2,5)
print(X)

In [ ]:
X+2

In [ ]:
X-2

In [ ]:
X*2

In [ ]:
X/2

In [ ]:
X**2

In [ ]:
Y = np.arange(2,12).reshape(2,5)
Y

In [ ]:
X+Y

In [ ]:
X-Y

In [ ]:
X*Y

In [ ]:
X/Y

In [ ]:
X**Y

## 比较运算

python的比较操作也都是universal function,只是返回的结果是bool值

In [ ]:
X>5

In [ ]:
X<5

In [ ]:
X==5

In [ ]:
X>=5

In [ ]:
X<=5

In [ ]:
2<=X<=6

## 其他的函数

numpy中还定义了其他的常用universal function具体可以看官方<https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs>

## 自定义ufunc


有的时候自带的ufunc不能满足需要,numpy允许自定义ufunc

### 使用接口`np.frompyfunc(func, nin, nout)`

例:用一个分段函数描述三角波

In [ ]:
def triangle_wave(x,c,c0,hc):
    """三角波"""
    x = x - int(x)
    if x >= c: r = 0.0
    elif x < c0: r = x / c0*hc
    else: r = (c-x) / (c-c0) * hc
    return r

In [ ]:
x = np.linspace(0,2,1000)
y1 = np.array([triangle_wave(t,0.6,0.4,1.0) for t in x])
triangl_ufunc1 = np.frompyfunc(triangle_wave,4,1)

In [ ]:
y2 = triangl_ufunc1(x,0.6,0.4,1.0)
plt.plot(range(len(y2)),y2)
plt.show()

In [ ]:
+ 使用接口`np.vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False, signature=None)`直接将python函数转化为ufunc

In [ ]:
triangl_ufunc2 = np.vectorize(triangle_wave)
triangl_ufunc2.__doc__

In [ ]:
y3 = triangl_ufunc2(x,0.6,0.4,1.0)
plt.plot(range(len(y3)),y3)
plt.show()

## ufunc的泛函

简单的可以理解为函数的函数就是泛函.numpy支持针对ufunc的泛函操作,具体的有如下几种:

### `at(a, indices, b=None)`

at用于指定下标执行函数,未被指定的就不会执行函数,注意其返回值为None,但它会改变第一个参数数组中对应元素的值.

In [ ]:
a = np.array([1, 2, 3, 4])
np.negative.at(a, [0, 1])
print(a)

In [ ]:
a = np.array([1, 2, 3, 4])
np.add.at(a,[0,1],1)
print(a)

### `reduce`

reduce和原生python中的reduce差不多,就是rfold,折叠操作的特化

In [ ]:
np.add.reduce([1,2,3]) # 相当于sum

In [ ]:
np.add.reduce([[1,2,3],[6,7,8]],axis=1) #可以指定延哪条轴折叠

### accumulate

accumulate和python3中functiontools新增的累积函数accumulate一样

In [ ]:
np.add.accumulate([1,2,3])

### outer

outer,会对俩数组中每两对元素组合进行运算.

In [ ]:
np.add.outer([1,2,3,4,5],[2,3,4])

### reduceat(a, indices, axis=0, dtype=None, out=None)

    指定范围执行reduce操作.相当于 ufunc.reduce(a[indices[i]:indices[i+1]]).

    特殊情况是--
    1. 如果i+1<i,那么直接去a的第i位即a[i]
    2. 最后一位时i+1 = a.shape[axis].

    在一维的情况下可以每两位组成一个范围对,然后通过切片操作隔位取值以过滤掉中间没有计算的部分

In [ ]:
x=np.arange(8)
print(x)

In [ ]:
x.shape

In [ ]:
np.add.reduceat(x,[0,4, 1,5, 2,6, 3,7])

在多维的情况下,可以用`axis`指定延哪条轴计算

In [ ]:
x = np.linspace(0, 15, 16).reshape(4,4)
print(x)
x.shape

In [ ]:
np.add.reduceat(x, [0, 3, 1, 2, 0])

## 指定轴执行函数

ufunc会在所有的元素上进行执行,但很多时候我们只希望指定轴执行函数.这时候就可以使用如下接口

### `np.apply_along_axis(func1d, axis, arr, *args, **kwargs)`

沿给定轴对一维切片应用函数,函数的第一个参数必须为一维数组.其他参数可以在后面放入

In [ ]:
def my_func(a,n):
    return (a[0] + a[-1]) * n
b = np.array([[1,2,3], [4,5,6], [7,8,9]])
np.apply_along_axis(my_func, 0, b,n=0.25)

### `np.apply_over_axes(func, a, axes)`

在多个轴上重复应用一个函数,函数的值为`res = func(a, axis)`

In [ ]:
a = np.arange(24).reshape(2,3,4)
print(a)
print(a.shape)

In [ ]:
np.apply_over_axes(np.sum, a, [0,2])

In [ ]:
np.sum(a,axis=0)